In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from rolling_ta.data import CSVLoader
from rolling_ta.volume import NumbaOBV
from rolling_ta.logging import logger
from ta.volume import OnBalanceVolumeIndicator
from time import time

In [3]:
xls_loader = CSVLoader()
data = xls_loader.read_resource().copy()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


In [4]:
# Precompile njit functions
NumbaOBV(data)

DEBUG - bytecode dump:
>          0	NOP(arg=None, lineno=297)
           2	RESUME(arg=0, lineno=297)
           4	LOAD_FAST(arg=0, lineno=307)
           6	LOAD_ATTR(arg=0, lineno=307)
          26	STORE_FAST(arg=3, lineno=307)
          28	LOAD_GLOBAL(arg=3, lineno=309)
          38	LOAD_ATTR(arg=4, lineno=309)
          58	LOAD_CONST(arg=1, lineno=309)
          60	LOAD_FAST(arg=3, lineno=309)
          62	CALL(arg=2, lineno=309)
          70	GET_ITER(arg=None, lineno=309)
>         72	FOR_ITER(arg=49, lineno=309)
          76	STORE_FAST(arg=4, lineno=309)
          78	LOAD_FAST(arg=0, lineno=310)
          80	LOAD_FAST(arg=4, lineno=310)
          82	BINARY_SUBSCR(arg=None, lineno=310)
          86	STORE_FAST(arg=5, lineno=310)
          88	LOAD_FAST(arg=0, lineno=311)
          90	LOAD_FAST(arg=4, lineno=311)
          92	LOAD_CONST(arg=1, lineno=311)
          94	BINARY_OP(arg=10, lineno=311)
          98	BINARY_SUBSCR(arg=None, lineno=311)
         102	STORE_FAST(arg=6, lineno=31

In [8]:
%%timeit
obv = OnBalanceVolumeIndicator(data["close"], data["volume"])
obv.on_balance_volume()

265 μs ± 5.74 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
%%timeit
NumbaOBV(data, memory=False)
    

51.7 μs ± 1.13 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
slice_a = data.copy().iloc[:100]
slice_b = data.copy().iloc[100:]

In [12]:
%%timeit
rolling = NumbaOBV(slice_a)
for _, series in slice_b.iterrows():
    rolling.update(series)

272 ms ± 3.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
